In [108]:
# getCV
"""
Loads CV data from a text file. Pulls the last curve out of the
repeats (red/ox cycle).
Returns structure. 

Example: mydata = getCV     (use UI to select file)

         mydata = getCV ('C:\Myname\CV\File.ras')   (directly offer filepath)

Onset potential methods: 
    1: Direct voltage at minimum cathodic peak
    2: Inflection point
    3: Intersection at baseline
    4: Fixed
"""
import os
import csv
import numpy as np

# def getFileList(directory = os.path.dirname(file_name)):#os.path.realpath(__file__)))
#     return os.listdir(directory)
# getFileList(filepath)

def getCV(fpath, area =1):
    cvdata = getcvdata(fpath)
    cvdata = getOnsetPotentials(cvdata)
    return cvdata

def getcvdata(filepath, area = 1, nheaderlines = 53):
    area = float(area)
    rawCV = {'voltage' : [],
             'current' : [],
             'cycle' : [] }

    with open(filepath,'r') as f:
        [next(f) for item in range(nheaderlines)] # Skips until the 54th line, where header line is. 
        f_reader = csv.DictReader(f, delimiter = '\t')    
        for row in f_reader:
            rawCV['voltage'].append(row['Ewe/V'])
            rawCV['current'].append(row['<I>/mA'])
            rawCV['cycle'].append(row['cycle number'])

    num_of_scans = np.unique(rawCV['cycle'])
    fs = num_of_scans[-1]
    final_scan = {'voltage' : [],
                 'current' : [],
                 'currentdensity' : np.array([])}

    j = 0  # This can probably be optimized. Final scan is the only important one.
    for n in rawCV['cycle']:
        if n == fs:
            final_scan['voltage'].append(rawCV['voltage'][j])
            final_scan['current'].append(rawCV['current'][j])
        j += 1
    
    final_scan['voltage'] = np.array(final_scan['voltage'], dtype = float)
    final_scan['current'] = np.array(final_scan['current'], dtype = float)
    final_scan['currentdensity'] = np.divide(final_scan['current'], area) # Gets current density
    return final_scan

def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)
    start = np.cumsum(a[:WSZ-1])[::2]/r
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

def getOnsetPotentials(cvdata,fitwidth = 40):
    #cvdata should be dict, output from getCV
    
    # Finding cathodic peak voltage
    X = cvdata['voltage']
    Y = cvdata['currentdensity']
    idx0 = np.argmin(Y)
    cvdata['cathodic_peak'] = X[idx0] 
    
    # Finding anodic peak current at the end of the voltammogram
    idx1 = np.argmax(X)
    cvdata['anodic_peak'] = Y[idx1] 

    # Inflection point onset
    idx2 = np.argmin(X)
    XX = X[idx1+500 : idx2 - 20]
    YY = Y[idx1+500 : idx2 - 20]
    
    dydx = np.gradient(YY)/np.gradient(XX) # Finds first derivative, delta y/ delta x
    dydx = smooth(dydx)
    
    idx3 = np.argmax(dydx) # Inflection point on the cathodic curve
    idx4 = np.argmin(abs(YY)) # Getting closest to the baseline center as possible
    cvdata['inflection_onset'] = XX[idx3]
    
    X_ = XX[idx3 - fitwidth//2: idx3 + fitwidth//2]
    Y_ = YY[idx3 - fitwidth//2: idx3 + fitwidth//2]
    linfit_3 = np.polyfit(X_,Y_,1)
    
    X_ = XX[idx4 - fitwidth//2: idx4 + fitwidth//2]
    Y_ = YY[idx4 - fitwidth//2: idx4 + fitwidth//2]
    linfit_4 = np.polyfit(X_,Y_,1)
    
    intersect_x = (linfit_4[1] - linfit_3[1]) / (linfit_3[0] - linfit_4[1])
    intersect_y = linfit_3[0] * intersect_x + linfit_3[1]
    cvdata['intersect_onset'] = [intersect_x, intersect_y]
    return cvdata


In [99]:
cvdata = getcvdata(filepath, area = .5)
print(cvdata)

{'voltage': array([0.19229576, 0.19345422, 0.19439143, ..., 0.19039437, 0.19128586,
       0.19239724]), 'current': array([0.00034713, 0.00034614, 0.00034744, ..., 0.00037424, 0.00037562,
       0.00040446]), 'currentdensity': array([0.00069426, 0.00069228, 0.00069488, ..., 0.00074847, 0.00075123,
       0.00080892])}


In [111]:
filepath = r"C:\Users\Skaggs\Desktop\Perovskites\2019_08_09 CV SnOx\10nm\180_11.4_FTO_02_CV_C02.txt"
getCV(filepath)

{'voltage': array([0.19229576, 0.19345422, 0.19439143, ..., 0.19039437, 0.19128586,
        0.19239724]),
 'current': array([0.00034713, 0.00034614, 0.00034744, ..., 0.00037424, 0.00037562,
        0.00040446]),
 'currentdensity': array([0.00034713, 0.00034614, 0.00034744, ..., 0.00037424, 0.00037562,
        0.00040446]),
 'cathodic_peak': -0.444778711,
 'anodic_peak': 0.010884792647427984,
 'inflection_onset': -0.309950382,
 'intersect_onset': [-0.18963104576730827, -0.00028764459757954497]}

In [112]:
len(_['voltage'])

4199